<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/web_scraping_on_colab_yahoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GoogleColab上にあるデフォルトパッケージの依存関係を整理
import pkg_resources
import importlib

importlib.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py'>

In [ ]:
# 環境構築 (スクレイピング)
%pip install -q google-colab-selenium

# 環境構築 (形態素解析)
import numpy
!pip install ginza==5.1.3 ja_ginza_electra==5.1.3 https://github.com/megagonlabs/ginza/releases/download/v5.1.3/ja_ginza_electra-5.1.3-with-model.tar.gz
!pip install "spacy>=3.2.0,<3.7.0"
!pip install "spacy[cuda117]"
!pip install fastapi==0.109.2 kaleido==0.2.1 python-multipart==0.0.9 uvicorn==0.27.1
!pip install japanize-matplotlib==1.1.3
!pip install pyvis==0.3.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.7/809.7 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.5/336.5 kB 45.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 35.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=91fd0395a3ba52bcf313a747ccc52dca93a15ae1ff0dedde875f8328fa868d53
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize-matplotlib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.7 MB/s eta 0:00:00


In [ ]:
# python検索パスを追加
import sys

# 追加したいパス
paths_to_add = [
    '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/financial_service_agency/'
]

for path in paths_to_add:
    if path not in sys.path:
        sys.path.append(path)

In [ ]:
#環境依存値設定
import os
os.environ['ENV'] = 'local'
os.environ['TARGET'] = 'Yahooファイナンス'
os.environ['COMPANY'] = '楽天グループ(株)'

from google.colab import drive
drive.mount('/content/drive')

print(sys.path)

Mounted at /content/drive
['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/financial_service_agency/']


In [ ]:
#標準ライブラリ
import os
import math
import json
import re
import time
from datetime import datetime

#内部ライブラリ
from path_config import WORKING_DIR
from env.get_env import APP_ENV
from config.get_config import APP_CONFIG
from utils.scraping_functions import scroll_page
from utils.nlp_utils import extract_trend_comment, classify_trend_word

#サードパーティライブラリ
import spacy
import pandas as pd
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from logging import (
    getLogger,
    StreamHandler,
    DEBUG,
    INFO,
    Formatter,
    config
)

In [ ]:
#環境値読み込み
TARGET_URL = APP_CONFIG['URL'][os.environ['TARGET']]
SAVE_FILE_NAME =  APP_CONFIG['SAVE_FILE_NAME'][os.environ['TARGET']]
DRIVER_WAIT_TIME = APP_CONFIG['DRIVER_WAIT_TIME']
MAX_SCROLL_ATTEMPTS = APP_CONFIG['MAX_SCROLL_ATTEMPTS']
SCROLL_PAUSE_TIME = APP_CONFIG['SCROLL_PAUSE_TIME']
result_dict = APP_CONFIG['RESULT_DICT']

#環境依存値読み込み
SUBMIT_FILE_PATH = APP_ENV['SUBMIT_FILE_PATH']

In [ ]:
# driver設定
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument(
       "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
})

driver = gs.Chrome(options=options)
driver.implicitly_wait(DRIVER_WAIT_TIME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#カスタムロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

try:
      driver.get(TARGET_URL)
      logger.debug(f'{TARGET_URL}：トップページへ移動しました')
except Exception as e:
      logger.error(f'{TARGET_URL}：トップページへ移動出来ませんでした: [e]')
      raise

2024-02-17 10:07:40,138 [DEBUG] main: https://finance.yahoo.co.jp/cm/message/1004755/3ze7/792：トップページへ移動しました


In [ ]:
# ページスクロール
scroll_page(driver, MAX_SCROLL_ATTEMPTS, SCROLL_PAUSE_TIME)

In [ ]:
ult_element = driver.find_element(By.ID,"cmtlst")
data_comment_elements = ult_element.find_elements(By.CSS_SELECTOR, 'li[data-comment]')
logger.info(f'コメント取得数:{len(data_comment_elements)}')

2024-02-17 10:11:08,589 [INFO] main: コメント取得数:3910


In [ ]:
script = """
var comments = [];
document.querySelectorAll('li[data-comment]').forEach(function(el) {
    var id = el.querySelector('.comNum').textContent.trim();
    var comment = el.querySelector('.comText').textContent.trim();
    var replyToElements = el.querySelectorAll('.comReplyTo');
    var replyToText = replyToElements.length > 0 ? replyToElements[0].textContent.trim() : '-';
    var postDate = el.querySelectorAll('.comWriter span a');
    var secondPostDateText = postDate.length > 0 ? postDate[0].textContent.trim() : '-';

    comments.push({id: id, comment: comment, replyTo: replyToText, postDate: secondPostDateText});
});
return comments;
"""

comments_data = driver.execute_script(script)


# JavaScriptから取得したデータを辞書に格納
cnt = 0
for item in comments_data:
    # 取得日時
    formatted_get_time = datetime.now().strftime('%Y/%m/%d %H:%M')

    # 投稿日時
    post_date_obj = datetime.strptime(item['postDate'], '%Y-%m-%d %H:%M')
    formatted_post_date = post_date_obj.strftime('%Y/%m/%d %H:%M')

    # アラート語句１
    # GPUを使用するようにspaCyに指示
    spacy.prefer_gpu()
    alert_word_1 = extract_trend_comment(item['comment'])[0]

    # 色分け
    trend_flag = classify_trend_word(alert_word_1)

    result_dict['銘柄'].append(os.environ['COMPANY'])
    result_dict['ID'].append(item['id'])
    result_dict['コメント原文'].append(item['comment'])
    result_dict['返信元ID'].append(item['replyTo'])
    result_dict['取得日時'].append(formatted_get_time)
    result_dict['投稿日時'].append(formatted_post_date)
    result_dict['アラート語句１'].append(alert_word_1)
    result_dict['みんかぶ上下'].append('-')
    result_dict['色分け'].append(trend_flag)
    result_dict['取得元'].append(os.environ['TARGET'])

    cnt += 1
    logger.info(f'取得数: {cnt}/{len(comments_data)}個取得完了')

2024-02-17 10:12:30,928 [INFO] main: 取得数: 1/3910個取得完了
2024-02-17 10:12:31,166 [INFO] main: 取得数: 2/3910個取得完了
2024-02-17 10:12:31,398 [INFO] main: 取得数: 3/3910個取得完了
2024-02-17 10:12:31,576 [INFO] main: 取得数: 4/3910個取得完了
2024-02-17 10:12:31,727 [INFO] main: 取得数: 5/3910個取得完了
2024-02-17 10:12:31,886 [INFO] main: 取得数: 6/3910個取得完了
2024-02-17 10:12:31,999 [INFO] main: 取得数: 7/3910個取得完了
2024-02-17 10:12:32,220 [INFO] main: 取得数: 8/3910個取得完了
2024-02-17 10:12:32,404 [INFO] main: 取得数: 9/3910個取得完了
2024-02-17 10:12:32,567 [INFO] main: 取得数: 10/3910個取得完了
2024-02-17 10:12:32,769 [INFO] main: 取得数: 11/3910個取得完了
2024-02-17 10:12:33,075 [INFO] main: 取得数: 12/3910個取得完了
2024-02-17 10:12:33,223 [INFO] main: 取得数: 13/3910個取得完了
2024-02-17 10:12:33,352 [INFO] main: 取得数: 14/3910個取得完了
2024-02-17 10:12:35,997 [INFO] main: 取得数: 15/3910個取得完了
2024-02-17 10:12:36,149 [INFO] main: 取得数: 16/3910個取得完了
2024-02-17 10:12:36,464 [INFO] main: 取得数: 17/3910個取得完了
2024-02-17 10:12:36,610 [INFO] main: 取得数: 18/3910個取得完了
2024-02-17 10:12:37

In [ ]:
import pandas as pd
df = pd.DataFrame(result_dict)
df.tail(100)

,銘柄,ID,コメント原文,返信元ID,取得日時,投稿日時,アラート語句１,みんかぶ上下,色分け,取得元
3810,楽天グループ(株),138,楽天は存続するよ！ 旨味があるから,-,02月17日 10:22,2月15日 07:23,-,-,-,Yahooファイナンス
3811,楽天グループ(株),137,もしかしたら将来的には楽天しか勝たん,-,02月17日 10:22,2月15日 07:23,-,-,-,Yahooファイナンス
3812,楽天グループ(株),136,楽天主要企業見習って決算3月締めにして欲しい\n何だって12月なんだよ,-,02月17日 10:22,2月15日 07:22,-,-,-,Yahooファイナンス
3813,楽天グループ(株),135,山登りが趣味だから楽天モバイルは使う気更々ないが、この優待を有効活用できないものかな?,-,02月17日 10:22,2月15日 07:21,-,-,-,Yahooファイナンス
3814,楽天グループ(株),134,優子リン 買うの？買わないの？どっちなんだい！,-,02月17日 10:22,2月15日 07:21,-,-,-,Yahooファイナンス
...,...,...,...,...,...,...,...,...,...,...
3905,楽天グループ(株),5,大量に株を持っている株主と100株のみ持っている株主とで株主優待に差をつけないというのはあり...,-,02月17日 10:23,2月15日 04:07,-,-,-,Yahooファイナンス
3906,楽天グループ(株),4,同意\n\n既存ユーザーにとってはどうでもいい内容\n\nだがそれでいい,>>,02月17日 10:23,2月15日 04:07,-,-,-,Yahooファイナンス
3907,楽天グループ(株),3,回線契約率伸びがあとすこし… \nそうか！株主に回線を使ってもらって契約率を上げよう！という...,-,02月17日 10:23,2月15日 04:05,-,-,-,Yahooファイナンス
3908,楽天グループ(株),2,そんな会社おもろくないやろ\nそんな思考の企業なら上場してる意味ないて,>>,02月17日 10:23,2月15日 04:02,-,-,-,Yahooファイナンス


In [ ]:
# エクセルファイルとして保存
SAVE_DIR = os.path.join(SUBMIT_FILE_PATH, SAVE_FILE_NAME)
df.to_excel(SAVE_DIR, header=True, index=False)

In [ ]:
# 参考
#コメント抽出
# ult_element = driver.find_element(By.ID,"cmtlst")
# data_comment_elements = ult_element.find_elements(By.CSS_SELECTOR, 'li[data-comment]')

# for data_comment_element in data_comment_elements:
#     # ID
#     id = data_comment_element.find_element(By.CSS_SELECTOR, ".comNum").text
#     # コメント
#     comment = data_comment_element.find_element(By.CSS_SELECTOR, ".comText").text
#     #返答先ID
#     reply_to_elements = data_comment_element.find_elements(By.CSS_SELECTOR, ".comReplyTo")
#     reply_to_text = reply_to_elements[0].text if reply_to_elements else '-'

#     #結果格納
#     result_dict['ID'].append(id)
#     result_dict['コメント'].append(comment)
#     result_dict['返信元ID'].append(reply_to_text)

# print(result_dict)

# # コメント列の内容をカンマで連結
# texts = df['コメント'].str.cat(sep=', ')
# logger.info(texts)

# エクセルからデータ読み込み
# 必要なライブラリのインストール
!pip install pandas openpyxl

# pandasをインポート
import pandas as pd

# エクセルファイルの読み込み
df = pd.read_excel(SAVE_DIR)